In [1]:
import os

In [2]:
%pwd

'd:\\Work\\Internship\\Inuron\\mushroom_classification\\test'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Work\\Internship\\Inuron\\mushroom_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    max_depth: int
    n_estimators: int
    TARGET_COLUMN: str

In [7]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path
            model_name=config.model_name
            max_depth=params.max_depth
            n_estimators=params.n_estimators
            TARGET_COLUMN=schema.TARGET_COLUMN
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from src import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]


        clf = RandomForestClassifier(max_depth=self.config.max_depth, n_estimators=self.config.n_estimators, random_state=42)
        clf.fit(train_x, train_y)

        joblib.dump(clf, os.path.join(self.config.root_dir, self.config.model_name))